In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Test_AChE_05.06"

/content/drive/MyDrive/Test_AChE_05.06


In [ ]:
!pip install catboost
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [1]:
from math import sqrt

import lightgbm as lgb
import pandas as pd
import deepchem as dc
import numpy as np
import warnings
from catboost import CatBoostRegressor
from deepchem.metrics import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/Users/thanhhung/anaconda3/envs/master/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


**Class**

In [2]:
class CollinearColumnRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold, col_regex=None, exclude_cols=None):
        self._threshold = threshold
        self._col_regex = col_regex
        self._exclude_cols = exclude_cols or []

    def _collinear_columns(self, df):
        if self._col_regex is None:
            df_sel = df.select_dtypes(include=[np.number, bool])
        else:
            df_sel = df.filter(regex=self._col_regex).select_dtypes(include=[np.number, bool])

        all_cols = [col for col in df_sel.columns if col not in self._exclude_cols]
        df_sel = df_sel[all_cols].astype("float64")

        corr_mat = df_sel.corr(method='pearson').abs()
        collin_cols = set()

        for i in range(len(all_cols) - 1):
            if all_cols[i] in collin_cols:
                continue

            for j in range(i + 1, len(all_cols)):
                if all_cols[j] in collin_cols:
                    continue

                corr = corr_mat.iloc[i, j]
                if corr > self._threshold:
                    collin_cols.add(all_cols[j])

        return collin_cols

    def fit(self, df):
        self._collin_cols = self._collinear_columns(df)
        return self

    def transform(self, df):
        nonexist_cols = [col for col in self._collin_cols if col not in df.columns]
        if len(nonexist_cols) > 0:
            warnings.warn("These collinear columns to be dropped do not exist in df: " + str(nonexist_cols))

        droped_cols = list(self._collin_cols.intersection(df.columns))
        warnings.warn("Number of columns dropped due to collinearity: " + str(len(droped_cols)))
        return df.drop(droped_cols, axis=1)


In [3]:
class NumImputer(BaseEstimator, TransformerMixin):
    def __init__(self, method="mean", exclude_cols=None):
        self._method = method
        self._exclude_cols = exclude_cols or []

    def fit(self, df_train):
        num_cols = df_train.select_dtypes(include=np.number).columns.tolist()
        num_cols = [col for col in num_cols if col not in self._exclude_cols]

        self._train_cols = df_train.columns.tolist()
        self._impute_values = {col: df_train[col].agg(self._method) for col in num_cols}

        return self

    def transform(self, df):
        df = df.copy()

        assert set(df.columns.tolist()) == set(
            self._train_cols), "Do not have the same set of columns as in training data"

        for col, val in self._impute_values.items():
            if df[col].isnull().sum() > 0:
                df[col].fillna(val, inplace=True)

        df = df.reindex(columns=self._train_cols)
        return df


class CatImputer(BaseEstimator, TransformerMixin):
    def __init__(self, val="MISSING"):
        self._val = val

    def fit(self, df_train):
        cat_cols = df_train.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
        self._train_cols = df_train.columns.tolist()
        self._impute_values = {col: self._val for col in cat_cols}

        return self

    def transform(self, df):
        df = df.copy()

        assert set(df.columns.tolist()) == set(
            self._train_cols), "Do not have the same set of columns as in training data"

        for col, val in self._impute_values.items():
            if df[col].isnull().sum() > 0:
                df[col].fillna(val, inplace=True)

        df = df.reindex(columns=self._train_cols)
        return df


In [4]:
class Standardizer(BaseEstimator, TransformerMixin):
    def __init__(self, exclude_cols=None):
        self._exclude_cols = exclude_cols or []

    def fit(self, df_train):
        num_cols = df_train.select_dtypes(include=np.number).columns.tolist()
        num_cols = [col for col in num_cols if col not in self._exclude_cols]

        self._mean = df_train[num_cols].mean()
        self._std = df_train[num_cols].std()

        return self

    def transform(self, df):
        df = df.copy()

        for col in self._mean.index:
            if col in df.columns and self._std[col] > 0:
                df[col] = (df[col] - self._mean[col]) / self._std[col]

        return df


**Prepare file**

In [5]:
#Load data
data_sht1 = pd.read_excel("data_dedup.xlsx", sheet_name="Sheet1")
data_sht1 = data_sht1.drop(["STT", "PUBCHEMCID"], axis=1)
data_sht1 = data_sht1.rename(columns={"SMILES": "smiles", "Gexp": "dG"})
data_sht1["code"] = "labeled_pubchem"

data_sht1["train_test"] = "train"
rnd = np.random.RandomState(42)
test_idx = rnd.choice(data_sht1.shape[0], size=162, replace=False)
data_sht1.loc[test_idx, 3] = "test"

data_sht1["smiles_len"] = data_sht1["smiles"].str.len()

print(data_sht1.shape)
data_sht1.head()

(762, 6)


,smiles,dG,code,train_test,3,smiles_len
0,C1=CC(=CC=C1O)O,-12.21,labeled_pubchem,train,NaN,15
1,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCO,-4.62,labeled_pubchem,train,NaN,34
2,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)O,-5.39,labeled_pubchem,train,test,43
3,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)OP(=...,-9.61,labeled_pubchem,train,NaN,52
4,CC1=C(SC=N1)CCO,-3.48,labeled_pubchem,train,NaN,15


In [6]:
chembl_27 = pd.read_table("chembl_27.smi", sep="\s+", header=None)
chembl_27 = chembl_27.rename(columns={0: "smiles"})

chembl_27["dG"] = np.nan
chembl_27["code"] = "chembl_27"
chembl_27["train_test"] = "pred"
chembl_27["smiles_len"] = chembl_27["smiles"].str.len()

chembl_27 = chembl_27[chembl_27["smiles_len"] <= 200]

print(chembl_27.shape)
chembl_27.head()

(145, 5)


,smiles,dG,code,train_test,smiles_len
0,C12=C(C=C(O1)C(C)(C)O)C1=C(C(=C2)O)C(=O)C=C(O1...,NaN,chembl_27,pred,58
1,C12=C(C=C(O1)C(C)(C)O)C1=C(C=C2)C(=O)C=C(O1)C1...,NaN,chembl_27,pred,55
2,C12=C(C=CO1)C1=C(C=C2)C(=O)C=C(O1)C1=CC=CC=C1,NaN,chembl_27,pred,45
3,C12=C(C=CO1)C1=C(C(=C2)O)C(=O)C=C(O1)C1=CC=CC=C1,NaN,chembl_27,pred,48
4,C12=CC3=C(C(=C1C=CO2)OC)C(=O)C=C(O3)C1=CC=CC=C1,NaN,chembl_27,pred,47


**Combine pdY**

In [7]:
pdY = pd.concat([data_sht1, chembl_27], axis=0, ignore_index=True)
pdY = pdY.reset_index()
pdY = pdY.rename(columns={"index": "id"})
pdY["id"] = pdY["id"].astype(str) + "_"
print(pdY.shape)

pdY.to_csv("pdY.csv", index=False)

pdY = pd.read_csv("pdY.csv")

(907, 7)


In [8]:
pdY[pdY["train_test"] == "train"].to_csv("pdY_train.csv", index=False)
pdY[pdY["train_test"] == "test"].to_csv("pdY_test.csv", index=False)

In [9]:
pdY[pdY["code"] == "chembl_27"].to_csv("pdY_chembl_27.csv", index=False)

**Extract RDKitDescriptors**

In [10]:
assert True

rdkit_featurizer = dc.feat.RDKitDescriptors()
X = rdkit_featurizer(pdY["smiles"])

X1 = []
for y in X:
    if y.shape[0] > 0:
        X1.append(y.tolist())
    else:
        y = [np.nan] * 200
        X1.append(y)
X1 = np.array(X1)

X2 = pd.DataFrame(X1, columns=rdkit_featurizer.descriptors)
X2["id"] = pdY["id"]
X2["smiles"] = pdY["smiles"]
X2["dG"] = pdY["dG"]
X2["code"] = pdY["code"]
X2["train_test"] = pdY["train_test"]
X2["smiles_len"] = pdY["smiles_len"]

X2.to_csv("pdXY_rdkit_descriptors_200ft.csv", index=False)

**Remove**

In [11]:
pdXY_200 = pd.read_csv("pdXY_rdkit_descriptors_200ft.csv")

In [12]:
PDY_COLS = ["id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_200.columns if col not in PDY_COLS]

In [13]:
pdXY_200_train = pdXY_200[pdXY_200["train_test"] == "train"].copy()

mostly_zero_cols = []
for col in PDX_COLS:
    zero_rate = (pdXY_200_train[col] == 0).mean()
    if zero_rate > 0.99:
        print("{}    {}".format(col, zero_rate))
        mostly_zero_cols.append(col)

pdXY_156 = pdXY_200.drop(mostly_zero_cols, axis=1)
print(pdXY_156.shape)

pdXY_156.to_csv("pdXY_rdkit_descriptors_156ft.csv", index=False)

NumRadicalElectrons    0.9960629921259843
SMR_VSA8    1.0
SlogP_VSA9    1.0
fr_Ar_COO    0.9960629921259843
fr_C_S    0.9973753280839895
fr_HOCCN    0.9960629921259843
fr_N_O    1.0
fr_SH    1.0
fr_aldehyde    0.9973753280839895
fr_azide    1.0
fr_barbitur    1.0
fr_benzodiazepine    1.0
fr_diazo    1.0
fr_dihydropyridine    1.0
fr_epoxide    1.0
fr_furan    0.9960629921259843
fr_hdrzine    0.9908136482939632
fr_hdrzone    0.9908136482939632
fr_imidazole    0.9921259842519685
fr_imide    0.9986876640419947
fr_isocyan    1.0
fr_isothiocyan    1.0
fr_lactam    1.0
fr_lactone    0.9960629921259843
fr_morpholine    0.9973753280839895
fr_nitro    0.9908136482939632
fr_nitro_arom    0.9908136482939632
fr_nitro_arom_nonortho    0.9908136482939632
fr_nitroso    0.9908136482939632
fr_oxazole    1.0
fr_phos_acid    0.994750656167979
fr_phos_ester    0.994750656167979
fr_priamide    0.994750656167979
fr_prisulfonamd    1.0
fr_sulfonamd    0.9921259842519685
fr_sulfone    1.0
fr_tetrazole    1.0
f

In [14]:
pdXY_156.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,MaxPartialCharge,...,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,code,train_test,smiles_len
0,8.645556,8.645556,0.169259,0.169259,0.490728,110.112,104.064,110.036779,42.0,0.115346,...,0.0,0.0,0.0,0.0,0_,C1=CC(=CC=C1O)O,-12.21,labeled_pubchem,train,15
1,8.975183,8.975183,0.177713,0.177713,0.794127,265.362,248.226,265.111759,96.0,0.224996,...,1.0,0.0,0.0,0.0,1_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCO,-4.62,labeled_pubchem,train,34
2,10.660372,10.660372,0.026376,-4.412252,0.520728,345.341,327.197,345.078089,120.0,0.469087,...,1.0,0.0,0.0,0.0,2_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)O,-5.39,labeled_pubchem,train,43
3,11.405303,11.405303,0.245126,-5.115401,0.351107,425.320,406.168,425.044420,144.0,0.480620,...,1.0,0.0,0.0,0.0,3_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)OP(=...,-9.61,labeled_pubchem,train,52
4,8.541389,8.541389,0.225231,0.225231,0.669277,143.211,134.139,143.040485,50.0,0.079707,...,1.0,0.0,0.0,0.0,4_,CC1=C(SC=N1)CCO,-3.48,labeled_pubchem,train,15


**Remove correlated columns**

In [15]:
pdXY_156 = pd.read_csv("pdXY_rdkit_descriptors_156ft.csv")
print(pdXY_156.shape)

pdXY_156_train = pdXY_156[pdXY_156["train_test"] == "train"].copy()
print(pdXY_156_train.shape)

remover = CollinearColumnRemover(0.95, exclude_cols=PDY_COLS)
remover.fit(pdXY_156_train)

pdXY_123 = remover.transform(pdXY_156)
print(pdXY_123.shape)


(907, 177)
(762, 177)
(907, 148)


/var/folders/06/l6_kydgj6_g6kmzzj27d8bt00000gn/T/ipykernel_79697/120061955.py:43: UserWarning: Number of columns dropped due to collinearity: 29
  warnings.warn("Number of columns dropped due to collinearity: " + str(len(droped_cols)))


In [16]:
pdXY_123.to_csv("pdXY_rdkit_descriptors_123ft.csv", index=False)

In [17]:
pdXY_123.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,...,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,code,train_test,smiles_len
0,8.645556,0.169259,0.169259,0.490728,110.112,0.115346,-0.507962,0.507962,0.115346,0.750000,...,0.0,0.0,0.0,0.0,0_,C1=CC(=CC=C1O)O,-12.21,labeled_pubchem,train,15
1,8.975183,0.177713,0.177713,0.794127,265.362,0.224996,-0.395996,0.395996,0.224996,1.444444,...,1.0,0.0,0.0,0.0,1_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCO,-4.62,labeled_pubchem,train,34
2,10.660372,0.026376,-4.412252,0.520728,345.341,0.469087,-0.383155,0.469087,0.383155,1.454545,...,1.0,0.0,0.0,0.0,2_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)O,-5.39,labeled_pubchem,train,43
3,11.405303,0.245126,-5.115401,0.351107,425.320,0.480620,-0.383155,0.480620,0.383155,1.307692,...,1.0,0.0,0.0,0.0,3_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)OP(=...,-9.61,labeled_pubchem,train,52
4,8.541389,0.225231,0.225231,0.669277,143.211,0.079707,-0.395999,0.395999,0.079707,1.777778,...,1.0,0.0,0.0,0.0,4_,CC1=C(SC=N1)CCO,-3.48,labeled_pubchem,train,15


**Impute missing and standardize**

In [18]:
pdXY_123 = pd.read_csv("pdXY_rdkit_descriptors_123ft.csv")
print("pdXY_123.shape", pdXY_123.shape)

PDY_COLS = ["id", "smiles", "dG", "code", "train_test", "smiles_len"]

imputer = NumImputer(method="median", exclude_cols=PDY_COLS)
imputer.fit(pdXY_123[pdXY_123["train_test"] == "train"].copy())
pdXY_123_clean = imputer.transform(pdXY_123)
print("pdXY_123_clean.shape", pdXY_123_clean.shape)

std = Standardizer(exclude_cols=PDY_COLS)
std.fit(pdXY_123_clean[pdXY_123_clean["train_test"] == "train"].copy())
pdXY_123_clean = std.transform(pdXY_123_clean)
print("pdXY_123_clean.shape", pdXY_123_clean.shape)

pdXY_123_clean.to_csv("pdXY_rdkit_descriptors_123ft_clean.csv", index=False)

pdXY_123.shape (907, 148)
pdXY_123_clean.shape (907, 148)
pdXY_123_clean.shape (907, 148)


In [19]:
pdXY_123_clean

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,...,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,code,train_test,smiles_len
0,-0.679753,-0.197613,0.563900,0.075558,-1.959877,-1.212874,-0.411363,0.338579,-1.451040,-0.846639,...,-0.187829,-0.109254,-0.467614,-0.176302,0_,C1=CC(=CC=C1O)O,-12.21,labeled_pubchem,train,15
1,-0.575245,-0.166639,0.571424,1.372029,-0.924144,-0.320614,0.387160,-0.407628,-0.332134,1.412428,...,5.317005,-0.109254,-0.467614,-0.176302,1_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCO,-4.62,labeled_pubchem,train,34
2,-0.040955,-0.721100,-3.513709,0.203753,-0.390574,1.665640,0.478738,0.079493,1.281776,1.445287,...,5.317005,-0.109254,-0.467614,-0.176302,2_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)O,-5.39,labeled_pubchem,train,43
3,0.195225,0.080344,-4.139521,-0.521063,0.142997,1.759487,0.478738,0.156355,1.281776,0.967565,...,5.317005,-0.109254,-0.467614,-0.176302,3_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)OP(=...,-9.61,labeled_pubchem,train,52
4,-0.712779,0.007455,0.613716,0.838523,-1.739061,-1.502881,0.387137,-0.407606,-1.814712,2.496780,...,5.317005,-0.109254,-0.467614,-0.176302,4_,CC1=C(SC=N1)CCO,-3.48,labeled_pubchem,train,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,0.787002,-0.704766,-0.065910,0.920322,-0.049973,-0.492138,-0.403730,0.331446,-0.547227,-0.145550,...,-0.187829,-0.109254,-0.467614,-0.176302,902_,C12=CC(=C3C(=C1C=CC(O2)(C)C)OC=C(C3=O)C1=C(C=C...,NaN,chembl_27,pred,62
903,-0.087639,-0.802676,0.065713,1.753854,-0.383942,-0.470739,-0.384840,0.313793,-0.520393,-0.163497,...,-0.187829,-0.109254,-0.467614,-0.176302,903_,C1(=CC2=C(C=C1O)[C@H]1[C@@H](CO2)C2=C(C(=C(C(=...,NaN,chembl_27,pred,62
904,0.773166,-0.636417,-0.037319,0.350678,-0.036523,-0.492239,-0.407169,0.334660,-0.547355,-0.033375,...,-0.187829,-0.109254,-0.467614,-0.176302,904_,C1(=C(C(=C2C(=C1)OC=C(C2=O)C1=C(C=C(C(=C1)OC)O...,NaN,chembl_27,pred,62
905,0.414622,-0.475675,-0.034387,1.149768,-0.878152,-0.736325,-0.411395,0.338608,-0.853441,-0.033375,...,-0.187829,-0.109254,-0.467614,-0.176302,905_,C1(=CC2=C(C(=C1)O)C(=O)C[C@H](O2)C1=CC=C(C=C1)O)O,NaN,chembl_27,pred,49


**Training_test**

In [20]:
def metrics(model, X, y):
    y_pred = model.predict(X)
    rmse = sqrt(mean_squared_error(y, y_pred))
    print('RMSE: %f' % rmse)
    mae = mean_absolute_error(y, y_pred)
    print('MAE: %f' % mae)
    r2 = r2_score(y, y_pred)
    print('R2: %f' % r2)
    pearson = pearsonr(y, y_pred)
    print('Pearson: %f' % pearson)

In [21]:
df = pd.read_csv('pdXY_rdkit_descriptors_200ft.csv')
# Extract rows where code is pred into another df
sync = df[df['train_test'] == 'pred']
# Remove rows where code is pred
df = df[df['train_test'] != 'pred']
# Drop train_test column
df = df.drop('train_test', axis=1)
df = df.drop('code', axis=1)
df.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,smiles_len
0,8.645556,8.645556,0.169259,0.169259,0.490728,110.112,104.064,110.036779,42.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0_,C1=CC(=CC=C1O)O,-12.21,15
1,8.975183,8.975183,0.177713,0.177713,0.794127,265.362,248.226,265.111759,96.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCO,-4.62,34
2,10.660372,10.660372,0.026376,-4.412252,0.520728,345.341,327.197,345.078089,120.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)O,-5.39,43
3,11.405303,11.405303,0.245126,-5.115401,0.351107,425.320,406.168,425.044420,144.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3_,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)OP(=...,-9.61,52
4,8.541389,8.541389,0.225231,0.225231,0.669277,143.211,134.139,143.040485,50.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,4_,CC1=C(SC=N1)CCO,-3.48,15


In [22]:
# Encode smiles column
le = LabelEncoder()
df['smiles'] = le.fit_transform(df['smiles'])

# remove _ at the end of column id and convert to int
df['id'] = df['id'].str.replace('_', '').astype(int)

# Split X and y where the dG is the target
X = df.drop('dG', axis=1)
y = df['dG']

# Split train test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.0583933 ,  0.0583933 , -0.15691098, ..., -0.82493055,
         0.01831198, -1.29653196],
       [ 0.56167447,  0.56167447, -0.59301214, ...,  1.00594357,
         0.0318679 ,  0.23367727],
       [ 1.22061204,  1.22061204, -0.50280046, ...,  0.25986237,
         1.54109301,  2.11974911],
       ...,
       [-0.30878601, -0.30878601,  0.70821929, ..., -0.49995039,
         0.91300232, -0.47804795],
       [ 1.25697854,  1.25697854, -0.49538598, ...,  0.25528518,
         1.52301846,  2.44002546],
       [ 0.44265924,  0.44265924, -0.34493566, ..., -1.26891753,
        -1.64906546, -0.40687543]])

In [24]:
y_train

199   -13.59
599    -9.83
436    -6.47
90     -2.46
371    -6.01
       ...  
71     -7.66
106    -5.48
270    -6.43
435    -6.41
102    -6.05
Name: dG, Length: 609, dtype: float64

In [25]:
# Check NaN values in X_train
pd.DataFrame(X_train).isnull().sum()
# Replace NaN values with 0
X_train = pd.DataFrame(X_train).fillna(0)
X_test = pd.DataFrame(X_test).fillna(0)

In [26]:
len(X_train)

609

In [27]:
len(X_test)

153

### Deep Learning

In [33]:
import torch
import torch.nn as nn


class ComplexNeuralNet(nn.Module):
    def __init__(self):
        super(ComplexNeuralNet, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.layer2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.layer3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.layer4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.layer5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        self.layer6 = nn.Linear(64, 32)
        self.bn6 = nn.BatchNorm1d(32)
        self.final_layer = nn.Linear(32, 1)
        self.leaky_relu = nn.LeakyReLU(0.01)
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.1)

    def forward(self, x):
        x = self.leaky_relu(self.bn1(self.layer1(x)))
        x = self.dropout1(x)
        x = self.leaky_relu(self.bn2(self.layer2(x)))
        x = self.dropout2(x)
        x = self.leaky_relu(self.bn3(self.layer3(x)))
        x = self.leaky_relu(self.bn4(self.layer4(x)))
        x = self.dropout3(x)
        x = self.leaky_relu(self.bn5(self.layer5(x)))
        x = self.leaky_relu(self.bn6(self.layer6(x)))
        x = self.final_layer(x)
        return x


# Ensure all datasets are numpy arrays (float32)
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32).reshape(-1, 1)
y_test = np.array(y_test, dtype=np.float32).reshape(-1, 1)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_tensor = torch.from_numpy(X_train).to(device)
X_test_tensor = torch.from_numpy(X_test).to(device)
y_train_tensor = torch.from_numpy(y_train).to(device)
y_test_tensor = torch.from_numpy(y_test).to(device)

# Model initialization
model = ComplexNeuralNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Evaluation on test data
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).cpu().numpy()
    test_rmse = sqrt(mean_squared_error(y_test, predictions))
    test_mae = mean_absolute_error(y_test, predictions)
    test_r2 = r2_score(y_test, predictions)
    test_pears = pearsonr(y_test.flatten(), predictions.flatten())
    print('Test RMSE:', test_rmse)
    print('Test R2:', test_r2)
    print('Test MAE:', test_mae)
    print('Test Pearson:', test_pears)


Epoch 0, Loss: 72.16734313964844
Epoch 100, Loss: 38.03135299682617
Epoch 200, Loss: 18.665931701660156
Epoch 300, Loss: 9.155315399169922
Epoch 400, Loss: 1.9421292543411255
Epoch 500, Loss: 0.26002994179725647
Epoch 600, Loss: 0.11210202425718307
Epoch 700, Loss: 0.07832922041416168
Epoch 800, Loss: 0.08287923783063889
Epoch 900, Loss: 0.06122889742255211
Test RMSE: 1.6241614305474772
Test R2: 0.44604407813653413
Test MAE: 1.0901718
Test Pearson: 0.7049791610417682


**XGBoost**

In [29]:
# Fit an XGBRegressor model
xgb = XGBRegressor(max_depth=100, n_estimators=2000, learning_rate=0.1, min_child_weight=1, subsample=0.8,
                   colsample_bytree=0.8, gamma=0.1, reg_alpha=0.1, reg_lambda=0.1)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=100, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=2000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
# Predict the test set and calculate the accuracy
metrics(xgb, X_test, y_test)

RMSE: 1.464800
MAE: 1.051102
R2: 0.549419
Pearson: 0.743066


In [59]:
X_test_decoded = scaler.inverse_transform(X_test)
X_test_decoded = pd.DataFrame(X_test_decoded, columns=X.columns)
X_test_decoded['smiles'] = le.inverse_transform(X_test_decoded['smiles'].astype(int))

In [60]:
X_test_decoded

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,smiles_len
0,9.130401,9.130401,0.000000,0.000000,0.490194,304.187,290.075,303.025876,92.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0,C1=CC=C2C(=C1)C=[N+](C3=CC=CC=C23)CCO.[Br-],43.0
1,15.665458,15.665458,0.326278,-1.859633,0.135020,834.584,820.472,834.114981,298.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,260.0,C1=CC2=C(C3=CC=C(N3)C(=C4C=CC(=N4)C(=C5C=CC(=N...,131.0
2,11.008855,11.008855,0.057600,-0.443464,0.768508,207.273,190.137,207.125929,82.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,CC(C)(C)C1=CC(=CC=C1)OC(=O)NC,29.0
3,12.626475,12.626475,0.041222,-0.041222,0.251963,464.650,424.330,464.303893,184.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,449.0,CCN(CC)CCCCCCCCOC1=CC2=C(C=C1)C(=O)C=C(O2)C3=C...,60.0
4,12.542041,12.542041,0.060146,-4.854756,0.171576,553.581,523.341,553.218841,210.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,595.0,C1CCC2=NC3=CC=CC=C3C(=C2C1)NCCCCCCNC(=O)C4=CC5...,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,11.155178,11.155178,0.286319,-2.934207,0.560997,219.153,207.057,219.041687,78.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,550.0,CC1=C(C=C[P+](O1)(O)OC)C(=O)OC,30.0
149,11.671774,11.671774,0.080529,-0.623183,0.663268,301.302,286.182,301.106256,114.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,382.0,CC(=O)OC1=CC=C(C=C1)COC(=O)NC2=C(C=NC=C2)N,42.0
150,12.562998,12.562998,0.086029,-0.255062,0.161782,467.589,436.341,467.232919,178.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,412.0,CC(=O)C1=CC=CC=C1NC(=O)C2=CC=C(C=C2)OCCCCCC[N+...,65.0
151,2.282488,2.282488,1.164675,1.164675,0.403329,312.501,280.245,312.255452,124.0,8.673617e-19,...,0.0,0.0,0.0,0.0,0.0,8.0,0.0,181.0,C1=CC=[N+](C=C1)CCCCCCCCCCC[N+]2=CC=CC=C2,41.0


**LinearRegression**

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Predict the test set and calculate the accuracy
metrics(lr, X_test, y_test)

RMSE: 4.356930
MAE: 2.126893
R2: -2.986375
Pearson: 0.222213


**CatBoost**

In [ ]:
cat = CatBoostRegressor()
cat.fit(X_train, y_train)

Learning rate set to 0.037857
0:	learn: 2.2942022	total: 34.5ms	remaining: 34.5s
1:	learn: 2.2661770	total: 57.6ms	remaining: 28.8s
2:	learn: 2.2437097	total: 83.2ms	remaining: 27.6s
3:	learn: 2.2206275	total: 110ms	remaining: 27.4s
4:	learn: 2.1991639	total: 131ms	remaining: 26.1s
5:	learn: 2.1832346	total: 152ms	remaining: 25.2s
6:	learn: 2.1656283	total: 174ms	remaining: 24.7s
7:	learn: 2.1448652	total: 198ms	remaining: 24.5s
8:	learn: 2.1231365	total: 219ms	remaining: 24.1s
9:	learn: 2.1045639	total: 246ms	remaining: 24.4s
10:	learn: 2.0828584	total: 268ms	remaining: 24.1s
11:	learn: 2.0660314	total: 300ms	remaining: 24.7s
12:	learn: 2.0512019	total: 322ms	remaining: 24.4s
13:	learn: 2.0332352	total: 344ms	remaining: 24.2s
14:	learn: 2.0214569	total: 365ms	remaining: 24s
15:	learn: 2.0050428	total: 390ms	remaining: 24s
16:	learn: 1.9908257	total: 413ms	remaining: 23.9s
17:	learn: 1.9753624	total: 439ms	remaining: 24s
18:	learn: 1.9579947	total: 461ms	remaining: 23.8s
19:	learn: 1.9

In [ ]:
# Predict the test set and calculate the accuracy
metrics(cat, X_test, y_test)

RMSE: 1.489504
MAE: 1.048928
R2: 0.534092
Pearson: 0.737660


**SVR**

In [ ]:
# Fit an SVR model
svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr.fit(X_train, y_train)

SVR(C=1000.0, gamma=0.1)

In [ ]:
# Predict the test set and calculate the accuracy
metrics(svr, X_test, y_test)

RMSE: 1.785138
MAE: 1.369094
R2: 0.330793
Pearson: 0.609301


**Random Forest**

In [ ]:
rf = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=100, n_estimators=1000, random_state=42)

In [ ]:
# Predict the test set and calculate the accuracy
metrics(rf, X_test, y_test)

RMSE: 1.480863
MAE: 1.059449
R2: 0.539482
Pearson: 0.744960


**Gradient Boosting**

In [ ]:
gb = GradientBoostingRegressor(n_estimators=1000, max_depth=100, random_state=42)
gb.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=100, n_estimators=1000, random_state=42)

In [ ]:
# Predict the test set and calculate the accuracy
metrics(gb, X_test, y_test)

RMSE: 2.063597
MAE: 1.383419
R2: 0.105734
Pearson: 0.592377


**K-Nearest Neighbors**

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsRegressor()

In [ ]:
# Predict the test set and calculate the accuracy
metrics(knn, X_test, y_test)

RMSE: 1.638135
MAE: 1.180588
R2: 0.436471
Pearson: 0.691400


**LightGBM**

In [ ]:
lgbm = lgb.LGBMRegressor()
lgbm.fit(X_train, y_train)

LGBMRegressor()

In [ ]:
metrics(lgbm, X_train, y_train)

RMSE: 0.309805
MAE: 0.194317
R2: 0.982089
Pearson: 0.991939


In [ ]:
# Predict the test set and calculate the accuracy
metrics(lgbm, X_test, y_test)

RMSE: 1.441519
MAE: 1.021037
R2: 0.563628
Pearson: 0.755566


**Generate dG for prediction data**

In [ ]:
sync = sync.drop('train_test', axis=1)
sync = sync.drop('code', axis=1)
sync.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,smiles_len
762,12.541088,12.541088,0.092849,-1.196134,0.574622,336.343,320.215,336.099774,126.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,762_,C12=C(C=C(O1)C(C)(C)O)C1=C(C(=C2)O)C(=O)C=C(O1...,NaN,58
763,12.480255,12.480255,0.108322,-1.108454,0.593099,320.344,304.216,320.104859,120.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,763_,C12=C(C=C(O1)C(C)(C)O)C1=C(C=C2)C(=O)C=C(O1)C1...,NaN,55
764,12.247806,12.247806,0.047960,-0.047960,0.515864,262.264,252.184,262.062994,96.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,764_,C12=C(C=CO1)C1=C(C=C2)C(=O)C=C(O1)C1=CC=CC=C1,NaN,45
765,12.308639,12.308639,0.138137,-0.281756,0.573006,278.263,268.183,278.057909,102.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,765_,C12=C(C=CO1)C1=C(C(=C2)O)C(=O)C=C(O1)C1=CC=CC=C1,NaN,48
766,12.547528,12.547528,0.138840,-0.138840,0.554940,292.290,280.194,292.073559,108.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,766_,C12=CC3=C(C(=C1C=CO2)OC)C(=O)C=C(O3)C1=CC=CC=C1,NaN,47


In [ ]:
# encode the SMILES strings to categorical values
le = LabelEncoder()
sync['smiles'] = le.fit_transform(sync['smiles'])

# remove _ at the end of column id and convert to int
sync['id'] = sync['id'].str.replace('_', '').astype(int)

# Split X and y where the dG is the target
X_sync = sync.drop('dG', axis=1)

# scale the data
scaler = StandardScaler()
X_sync = scaler.fit_transform(X_sync)

# Predict the sync set with the best model
y_sync = cat.predict(X_sync)
y_sync

array([ -8.17694479,  -8.41788454,  -7.21443535,  -7.72561154,
        -7.98454906,  -9.00279854,  -8.9963678 ,  -9.67745418,
        -8.94471624,  -9.60939004,  -7.39387189,  -8.62968858,
        -8.86970492,  -8.73340676,  -8.5303427 ,  -7.92456941,
        -7.81691671,  -7.54763013,  -7.7282909 ,  -8.35819809,
        -8.45289038,  -7.64081995,  -7.86166226,  -7.79064893,
        -7.82584455,  -8.72757795,  -8.50452225,  -8.26914623,
        -8.32376375,  -8.31867653,  -9.15877003,  -8.23273542,
        -8.09986495,  -8.10978274,  -8.98398373,  -8.67447734,
        -8.42911328,  -8.31453159,  -8.46855922,  -9.3991156 ,
        -9.56622277,  -8.6603098 ,  -7.50500338,  -7.58412789,
        -8.59167634,  -9.24100536,  -8.11969693,  -8.31896013,
        -8.2420528 ,  -9.01603302,  -8.82019182,  -8.3905155 ,
        -7.88778279,  -7.89076339,  -7.81317705,  -8.39887777,
        -7.64921003,  -9.21871915,  -8.6494609 ,  -7.26456248,
        -7.66240614,  -8.43336357,  -8.21782603,  -9.40

In [ ]:
# Add dG column to sync df
sync['dG'] = y_sync

In [ ]:
sync.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,id,smiles,dG,smiles_len
762,12.541088,12.541088,0.092849,-1.196134,0.574622,336.343,320.215,336.099774,126.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,762,82,-8.176945,58
763,12.480255,12.480255,0.108322,-1.108454,0.593099,320.344,304.216,320.104859,120.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,763,83,-8.417885,55
764,12.247806,12.247806,0.047960,-0.047960,0.515864,262.264,252.184,262.062994,96.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,764,88,-7.214435,45
765,12.308639,12.308639,0.138137,-0.281756,0.573006,278.263,268.183,278.057909,102.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,765,87,-7.725612,48
766,12.547528,12.547528,0.138840,-0.138840,0.554940,292.290,280.194,292.073559,108.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,766,100,-7.984549,47


In [ ]:
sync = sync[['id', 'smiles', 'dG']]

In [ ]:
sync.to_csv('result_delta_g.csv', index=False)